# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [19]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score
import xlsxwriter
from secrets import IEX_CLOUD_API_TOKEN

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [3]:
#stks = pd.read_excel('trades.xlsx')
#stks

In [4]:
stocks = pd.read_csv('sp_500_stocks.csv')

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [5]:
symbol = 'AAPL'
api = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2280457701072,
 'week52high': 145.93,
 'week52low': 54.79,
 'week52highSplitAdjustOnly': 148.09,
 'week52lowSplitAdjustOnly': 53.46,
 'week52change': 0.6340437974915356,
 'sharesOutstanding': 16860762196,
 'float': 0,
 'avg10Volume': 81426997,
 'avg30Volume': 100804097,
 'day200MovingAvg': 123.47,
 'day50MovingAvg': 135.6,
 'employees': 0,
 'ttmEPS': 3.7,
 'ttmDividendRate': 0.8375640647492273,
 'dividendYield': 0.006317878726666205,
 'nextDividendDate': '0',
 'exDividendDate': '2021-01-30',
 'nextEarningsDate': '2021-01-17',
 'peRatio': 35.20499086998211,
 'beta': 1.1748225627207107,
 'maxChangePercent': 49.61177542543205,
 'year5ChangePercent': 4.856047549693012,
 'year2ChangePercent': 2.1750936845827247,
 'year1ChangePercent': 0.665135637386341,
 'ytdChangePercent': -0.021809629649275224,
 'month6ChangePercent': 0.1309816745219626,
 'month3ChangePercent': 0.10221971992681729,
 'month1ChangePercent': 0.02262662617054473,
 'day30ChangePercent

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [6]:
data['year1ChangePercent']

0.665135637386341

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [7]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [8]:
df = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats,quote,news,chart&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        df = df.append(pd.Series([symbol,data[symbol]['price'],data[symbol]['stats']['year1ChangePercent'],'N/A'],index=my_columns),ignore_index=True)
        
df    

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,129.01,0.526079,N/A
1,AAL,18.43,-0.400248,N/A
2,AAP,174.00,0.200157,N/A
3,AAPL,132.42,0.658352,N/A
4,ABBV,108.81,0.203111,N/A
...,...,...,...,...
500,YUM,106.93,0.033118,N/A
501,ZBH,157.68,-0.007745,N/A
502,ZBRA,491.63,1.052433,N/A
503,ZION,50.79,0.156894,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [9]:
df.sort_values('One-Year Price Return', ascending=False, inplace=True)
df = df[:50]
df.reset_index(inplace = True)
df

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,78,CARR,36.74,2.121795,N/A
1,179,FCX,34.64,1.924385,N/A
2,387,PYPL,292.80,1.383297,N/A
3,24,ALGN,619.63,1.261023,N/A
4,275,LB,50.95,1.063207,N/A
5,345,NVDA,608.06,1.0528,N/A
6,502,ZBRA,491.63,1.052433,N/A
7,410,SIVB,541.76,1.014207,N/A
8,266,KLAC,327.89,0.984813,N/A
9,236,IDXX,564.89,0.948161,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [10]:
position_size = float(input('Investment')) / len(df.index)
df['Number of Shares to Buy'] = df.apply(lambda x: math.floor(position_size / x['Price']),axis=1)
df

Investment10000000


<ipython-input-10-5323145526f1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Number of Shares to Buy'] = df.apply(lambda x: math.floor(position_size / x['Price']),axis=1)


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,78,CARR,36.74,2.121795,5443
1,179,FCX,34.64,1.924385,5773
2,387,PYPL,292.80,1.383297,683
3,24,ALGN,619.63,1.261023,322
4,275,LB,50.95,1.063207,3925
5,345,NVDA,608.06,1.0528,328
6,502,ZBRA,491.63,1.052433,406
7,410,SIVB,541.76,1.014207,369
8,266,KLAC,327.89,0.984813,609
9,236,IDXX,564.89,0.948161,354


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [14]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile'
]
hqm_dataframe = pd.DataFrame(columns=hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats,quote,news,chart&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['price'],
                'N/A',
                data[symbol]['stats']['year1ChangePercent'],
                'N/A',
                data[symbol]['stats']['month6ChangePercent'],
                'N/A',
                data[symbol]['stats']['month3ChangePercent'],
                'N/A',
                data[symbol]['stats']['month1ChangePercent'],
                'N/A'
            ],
            index=hqm_columns),
            ignore_index=True
        )

hqm_dataframe

,Ticker,Price,Number of Shares to buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile
0,A,133.58,N/A,0.533581,N/A,0.313614,N/A,0.207809,N/A,0.011685,N/A
1,AAL,18.07,N/A,-0.393944,N/A,0.42017,N/A,0.399295,N/A,0.12737,N/A
2,AAP,172.20,N/A,0.199034,N/A,0.032291,N/A,0.138228,N/A,0.014292,N/A
3,AAPL,133.47,N/A,0.668221,N/A,0.129054,N/A,0.104647,N/A,0.021885,N/A
4,ABBV,107.23,N/A,0.194288,N/A,0.13924,N/A,0.089628,N/A,-0.041321,N/A
...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,106.01,N/A,0.033091,N/A,0.129449,N/A,0.021773,N/A,-0.007148,N/A
501,ZBH,158.05,N/A,-0.007708,N/A,0.134303,N/A,0.062541,N/A,-0.021162,N/A
502,ZBRA,490.96,N/A,1.064781,N/A,0.695729,N/A,0.43047,N/A,0.174648,N/A
503,ZION,52.50,N/A,0.156245,N/A,0.564089,N/A,0.305085,N/A,0.025691,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [20]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])
        
hqm_dataframe

TypeError: '<' not supported between instances of 'NoneType' and 'float'

## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

## Saving Our Excel Output

As before, saving our Excel output is very easy: